## Setup

In [1]:
from pathlib import Path
import duckdb

# Import our refactored modules
from morpho_blue.data.parquet_repository import ParquetRepository
from morpho_blue.indicator_service import IndicatorService

## Configure Data Access

In [2]:
# Market ID for cbBTC market (from your example)
CBBTC_MARKET_ID = "0x9103c3b4e834476c9a62ea009ba2c884ee42e94e6e314a26f04d312434191836"

# Data directory
DATA_ROOT = Path.cwd().parent / "data_examples"

print(f"Data root: {DATA_ROOT}")
print(f"Market ID: {CBBTC_MARKET_ID}")

Data root: /home/youssef/morpho_blue/data_examples
Market ID: 0x9103c3b4e834476c9a62ea009ba2c884ee42e94e6e314a26f04d312434191836


In [3]:
# Initialize DuckDB connection
con = duckdb.connect()

# Initialize repository
repo = ParquetRepository(con=con, root=str(DATA_ROOT))

# Initialize service
service = IndicatorService(repo=repo, con=con)

print("✓ Service initialized")

✓ Service initialized


## Build Market Dataset

The `IndicatorService` orchestrates the entire pipeline:
1. Fetches all event types from parquet files
2. Standardizes events into a unified ledger
3. Enriches with cumulative states
4. Computes advanced indicators and features

In [4]:
# Build complete indicator dataset
indicators = service.build_market_dataset(market_id=CBBTC_MARKET_ID)

print(f"Dataset shape: {indicators.num_rows} rows × {indicators.num_columns} columns")
print(f"\nSchema fields ({len(indicators.schema)}):\n")
for field in indicators.schema:
    print(f"  - {field.name}: {field.type}")

Dataset shape: 4084573 rows × 59 columns

Schema fields (59):

  - market_id: string
  - block_number: int64
  - log_index: int64
  - block_timestamp: int64
  - tx_hash: string
  - event_type: string
  - delta_supply_assets: int64
  - delta_borrow_assets: int64
  - delta_collateral_assets: int64
  - total_supplied_assets: int64
  - outstanding_borrow_assets: int64
  - total_collateral_assets: int64
  - borrow_rate_per_sec: double
  - utilization_rate: double
  - borrow_apy: double
  - supply_rate_per_sec: double
  - supply_apy: double
  - delta_utilization: double
  - delta_borrow_apy: double
  - delta_supply_apy: double
  - borrow_apr: double
  - borrow_in_assets: double
  - borrow_out_assets: double
  - supply_in_assets: double
  - supply_out_assets: double
  - interest_assets: double
  - util_mean_5min: double
  - util_std_5min: double
  - util_mean_1h: double
  - util_std_1h: double
  - util_mean_6h: double
  - util_std_6h: double
  - borrow_apr_mean_5min: double
  - borrow_apr_std

## Explore Data

Let's examine the indicator dataset structure and content.

In [5]:
# Show first few rows
print("First 5 rows:")
print(indicators.slice(0, 5).to_pandas())

First 5 rows:
                                           market_id  block_number  log_index  \
0  0x9103c3b4e834476c9a62ea009ba2c884ee42e94e6e31...      19403244        265   
1  0x9103c3b4e834476c9a62ea009ba2c884ee42e94e6e31...      19713959         49   
2  0x9103c3b4e834476c9a62ea009ba2c884ee42e94e6e31...      19713959         50   
3  0x9103c3b4e834476c9a62ea009ba2c884ee42e94e6e31...      19714179        151   
4  0x9103c3b4e834476c9a62ea009ba2c884ee42e94e6e31...      19714179        158   

   block_timestamp                                            tx_hash  \
0       1725595835  0x808559ab9f6195db59cac4cb8205cbfd617ca6eb7013...   
1       1726217265  0x4b95f45c3afdbdb97aca807ddc82d4d2b7cd2c09801b...   
2       1726217265  0x4b95f45c3afdbdb97aca807ddc82d4d2b7cd2c09801b...   
3       1726217705  0xbc8e2030283aef18ec38474c5a6f62dbb20bc9d18688...   
4       1726217705  0xbc8e2030283aef18ec38474c5a6f62dbb20bc9d18688...   

         event_type  delta_supply_assets  delta_borrow_asset

In [6]:
# Basic statistics
print("Event type distribution:")
event_counts = indicators.group_by('event_type').aggregate([
    ('event_type', 'count')
])
print(event_counts.to_pandas())

Event type distribution:
           event_type  event_type_count
0    SupplyCollateral            161073
1      AccrueInterest           1608168
2              Borrow            162458
3              Supply           1147409
4            Withdraw            850012
5               Repay            110916
6  WithdrawCollateral             41076
7           Liquidate              3461


## Inspect Key Metrics

In [7]:
# Get latest state
latest = indicators.slice(indicators.num_rows - 1, 1)

print("Latest Market State:")
print(f"  Block: {latest['block_number'][0].as_py():,}")
print(f"  Total Supplied: {latest['total_supplied_assets'][0].as_py():,}")
print(f"  Outstanding Borrows: {latest['outstanding_borrow_assets'][0].as_py():,}")
print(f"  Total Collateral: {latest['total_collateral_assets'][0].as_py():,}")
print(f"  Utilization Rate: {latest['utilization_rate'][0].as_py():.2%}")
print(f"  Borrow APY: {latest['borrow_apy'][0].as_py():.2%}")
print(f"  Supply APY: {latest['supply_apy'][0].as_py():.2%}")

Latest Market State:
  Block: 39,857,315
  Total Supplied: 1,141,017,431,824,926
  Outstanding Borrows: 1,006,608,288,161,185
  Total Collateral: 2,123,271,042,107
  Utilization Rate: 88.22%
  Borrow APY: 6.14%
  Supply APY: 5.40%


## Query with DuckDB

We can use DuckDB to run SQL queries on the PyArrow table.

In [8]:
# Register the table in DuckDB
con.register('indicators', indicators)

# Query: Average utilization by event type
query = """
SELECT 
    event_type,
    COUNT(*) as event_count,
    AVG(utilization_rate) as avg_utilization,
    AVG(borrow_apy) as avg_borrow_apy,
    AVG(supply_apy) as avg_supply_apy
FROM indicators
GROUP BY event_type
ORDER BY event_count DESC
"""

result = con.execute(query).fetch_arrow_table()
print(result.to_pandas())

           event_type  event_count  avg_utilization  avg_borrow_apy  \
0      AccrueInterest      1608168         0.889145        0.068981   
1              Supply      1147409         0.886666        0.069349   
2            Withdraw       850012         0.885331        0.068481   
3              Borrow       162458         0.889128        0.066982   
4    SupplyCollateral       161073         0.889241        0.067469   
5               Repay       110916         0.890436        0.068886   
6  WithdrawCollateral        41076         0.888535        0.068889   
7           Liquidate         3461         0.896853        0.074951   

   avg_supply_apy  
0        0.061267  
1        0.061414  
2        0.060541  
3        0.059507  
4        0.059956  
5        0.061274  
6        0.061201  
7        0.067028  


## Time Series Analysis

In [9]:
# Query: Utilization over time (hourly aggregation)
query = """
SELECT 
    epoch_ms(block_timestamp * 1000) as timestamp,
    utilization_rate,
    borrow_apy,
    supply_apy,
    total_supplied_assets,
    outstanding_borrow_assets
FROM indicators
ORDER BY block_number, log_index
LIMIT 100
"""

timeseries = con.execute(query).fetch_arrow_table()
print(f"Time series data: {timeseries.num_rows} rows")
print(timeseries.slice(0, 10).to_pandas())

Time series data: 100 rows
            timestamp  utilization_rate  borrow_apy    supply_apy  \
0 2024-09-06 04:10:35          0.000000    0.008940  0.000000e+00   
1 2024-09-13 08:47:45          0.000000    0.005123  0.000000e+00   
2 2024-09-13 08:47:45          0.000000    0.005123  0.000000e+00   
3 2024-09-13 08:55:05          0.000000    0.005123  0.000000e+00   
4 2024-09-13 08:55:05          0.000000    0.002958  0.000000e+00   
5 2024-09-13 08:55:05          0.000000    0.002958  0.000000e+00   
6 2024-09-13 08:55:05          0.983789    0.002958  2.910250e-03   
7 2024-09-13 15:36:15          0.983789    0.002958  2.910250e-03   
8 2024-09-13 15:36:15          0.983790    0.043075  4.236237e-02   
9 2024-09-13 15:36:15          0.000007    0.043075  2.757488e-07   

   total_supplied_assets  outstanding_borrow_assets  
0                      0                          0  
1                      0                          0  
2                1000000                          0

## Inspect Individual Pipeline Stages

Let's examine each stage of the transformation pipeline separately.

In [10]:
# Stage 1: Fetch raw events
from morpho_blue.domain.market_events import MarketEventTables

events = MarketEventTables(
    market_id=CBBTC_MARKET_ID,
    supplies=repo.read_event(event="Supply", market_id=CBBTC_MARKET_ID),
    withdraws=repo.read_event(event="Withdraw", market_id=CBBTC_MARKET_ID),
    borrows=repo.read_event(event="Borrow", market_id=CBBTC_MARKET_ID),
    repays=repo.read_event(event="Repay", market_id=CBBTC_MARKET_ID),
    liquidates=repo.read_event(event="Liquidate", market_id=CBBTC_MARKET_ID),
    accrue_interests=repo.read_event(event="AccrueInterest", market_id=CBBTC_MARKET_ID),
    collateral_supplies=repo.read_event(event="SupplyCollateral", market_id=CBBTC_MARKET_ID),
    collateral_withdraws=repo.read_event(event="WithdrawCollateral", market_id=CBBTC_MARKET_ID),
)

print("Raw Events Loaded:")
print(f"  Supplies: {events.supplies.num_rows}")
print(f"  Withdraws: {events.withdraws.num_rows}")
print(f"  Borrows: {events.borrows.num_rows}")
print(f"  Repays: {events.repays.num_rows}")
print(f"  Liquidates: {events.liquidates.num_rows}")
print(f"  AccrueInterests: {events.accrue_interests.num_rows}")
print(f"  SupplyCollateral: {events.collateral_supplies.num_rows}")
print(f"  WithdrawCollateral: {events.collateral_withdraws.num_rows}")

Raw Events Loaded:
  Supplies: 1147409
  Withdraws: 850012
  Borrows: 162458
  Repays: 110916
  Liquidates: 3461
  AccrueInterests: 1608168
  SupplyCollateral: 161073
  WithdrawCollateral: 41076


In [11]:
# Stage 2: Standardize into ledger
from morpho_blue.transformations import build_market_ledger_raw

raw_ledger = build_market_ledger_raw(events=events)

print(f"Raw Ledger: {raw_ledger.table.num_rows} rows")
print(f"\nSchema: {raw_ledger.schema}")
print("\nFirst 5 rows:")
print(raw_ledger.table.slice(0, 5).to_pandas())

Raw Ledger: 4084573 rows

Schema: market_id: string not null
block_number: int64 not null
log_index: int64 not null
block_timestamp: int64 not null
tx_hash: string not null
event_type: string not null
delta_supply_assets: int64 not null
delta_borrow_assets: int64 not null
delta_collateral_assets: int64 not null
borrow_rate_per_sec: double

First 5 rows:
                                           market_id  block_number  log_index  \
0  0x9103c3b4e834476c9a62ea009ba2c884ee42e94e6e31...      19403244        265   
1  0x9103c3b4e834476c9a62ea009ba2c884ee42e94e6e31...      19713959         49   
2  0x9103c3b4e834476c9a62ea009ba2c884ee42e94e6e31...      19713959         50   
3  0x9103c3b4e834476c9a62ea009ba2c884ee42e94e6e31...      19714179        151   
4  0x9103c3b4e834476c9a62ea009ba2c884ee42e94e6e31...      19714179        158   

   block_timestamp                                            tx_hash  \
0       1725595835  0x808559ab9f6195db59cac4cb8205cbfd617ca6eb7013...   
1       172

In [12]:
# Stage 3: Enrich with states
from morpho_blue.transformations import enrich_market_ledger_with_states

ledger = enrich_market_ledger_with_states(con, raw_ledger=raw_ledger)

print(f"Enriched Ledger: {ledger.table.num_rows} rows")
print(f"\nNew fields added:")
raw_fields = set(f.name for f in raw_ledger.schema)
new_fields = [f.name for f in ledger.schema if f.name not in raw_fields]
for field in new_fields:
    print(f"  - {field}")

Enriched Ledger: 4084573 rows

New fields added:
  - total_supplied_assets
  - outstanding_borrow_assets
  - total_collateral_assets
  - utilization_rate
  - borrow_apy
  - supply_rate_per_sec
  - supply_apy
  - delta_utilization
  - delta_borrow_apy
  - delta_supply_apy


In [13]:
# Stage 4: Compute indicators
from morpho_blue.transformations import compute_market_indicators

indicators_detailed = compute_market_indicators(ledger=ledger)

print(f"Indicators: {indicators_detailed.table.num_rows} rows")
print(f"\nIndicator fields added:")
ledger_fields = set(f.name for f in ledger.schema)
indicator_fields = [f.name for f in indicators_detailed.schema if f.name not in ledger_fields]
for field in indicator_fields:
    print(f"  - {field}")

Indicators: 4084573 rows

Indicator fields added:
  - borrow_apr
  - borrow_in_assets
  - borrow_out_assets
  - supply_in_assets
  - supply_out_assets
  - interest_assets
  - util_mean_5min
  - util_std_5min
  - util_mean_1h
  - util_std_1h
  - util_mean_6h
  - util_std_6h
  - borrow_apr_mean_5min
  - borrow_apr_std_5min
  - borrow_apr_mean_1h
  - borrow_apr_std_1h
  - borrow_apr_mean_6h
  - borrow_apr_std_6h
  - borrow_intensity_5min
  - repay_intensity_5min
  - supply_intensity_5min
  - withdraw_intensity_5min
  - interest_intensity_5min
  - borrow_intensity_1h
  - repay_intensity_1h
  - supply_intensity_1h
  - withdraw_intensity_1h
  - interest_intensity_1h
  - borrow_intensity_6h
  - repay_intensity_6h
  - supply_intensity_6h
  - withdraw_intensity_6h
  - interest_intensity_6h
  - delta_borrow_apr
  - elasticity_apr_per_util
  - regime_util_gt_90
  - regime_util_gt_95
  - regime_util_gt_99
  - borrow_apr_6h_avg_ui
